<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_MLP/blob/main/%5BTEST%5DDNN_AR_tf2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)

#Data Load
df_train = pd.read_csv('/dir')
df_test = pd.read_csv('/dir')
ar_train_data = df_train
ar_test_data = df_test
testify = len(ar_test_data)/2
testify = np.int(testify)
print(testify)
ar_train_data_X = df_train.iloc[:len(ar_train_data)-1:,[0,1,3,4]]
ar_train_data_Y = df_train.iloc[1:len(ar_train_data):,-3]
ar_test_data_X = df_test.iloc[:len(ar_test_data)-1:,[0,1,3,4]]
ar_test_data_Y = df_test.iloc[1:len(ar_test_data):,-3]

print(ar_train_data_X)
print(ar_train_data_Y)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

In [ ]:
# Data Preprocessing
trainX_max = np.max(ar_train_data_X)
trainY_max = np.max(ar_train_data_Y)
trainX_min = np.min(ar_train_data_X)
trainY_min = np.min(ar_train_data_Y)

testX_max = np.max(ar_test_data_X)
testY_max = np.max(ar_test_data_Y)
testX_min = np.min(ar_test_data_X)
testY_min = np.min(ar_test_data_Y)

ar_test_data_X_testify_original = ar_test_data_X
ar_test_data_Y_testify_original = ar_test_data_Y

print(ar_test_data_X)
ar_train_data_X = MinMaxScaler(ar_train_data_X)
ar_train_data_Y = MinMaxScaler(ar_train_data_Y)

ar_test_data_X = MinMaxScaler(ar_test_data_X)
ar_test_data_Y = MinMaxScaler(ar_test_data_Y)

testify = len(ar_test_data)/2
testify = np.int(testify)
ar_test_data_X_testify = ar_test_data_X[:testify]
ar_test_data_Y_testify = ar_test_data_Y[testify:]

print(ar_test_data_X_testify.shape)
print(ar_test_data_X)
print(ar_test_data_Y_testify.shape)
print(ar_test_data_Y)
print("TestX_max : ",testX_max)
print("TestX_min : ",testX_min)


In [ ]:
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
hiddenDim = 10
input_dim = 4
Dimension = 1
lr = 0.001
iterations = 10
batch_size = 16

model = keras.Sequential()
model.add(layers.Dense(units = hiddenDim, activation='sigmoid',input_dim = input_dim, kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'sigmoid', kernel_initializer = initializer))
model.add(layers.Dense(Dimension, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()

model1 = keras.Sequential()
model1.add(layers.Dense(units = hiddenDim, activation='relu',input_dim = input_dim, kernel_initializer = initializer))
model1.add(layers.Dense(units = hiddenDim, activation = 'relu', kernel_initializer = initializer))
model1.add(layers.Dense(Dimension, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model1.compile(optimizer=adam, loss='mse')
model1.summary()

model2 = keras.Sequential()
model2.add(layers.Dense(units = hiddenDim, activation='linear',input_dim = input_dim, kernel_initializer = initializer))
model2.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model2.add(layers.Dense(Dimension, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model2.compile(optimizer=adam, loss='mse')
model2.summary()


In [ ]:
# Training
hist = model.fit(ar_train_data_X, ar_train_data_Y, epochs = iterations, batch_size = batch_size)
hist1 = model1.fit(ar_train_data_X, ar_train_data_Y, epochs = iterations, batch_size = batch_size)
hist2 = model2.fit(ar_train_data_X, ar_train_data_Y, epochs = iterations, batch_size = batch_size)

In [ ]:
# Evaluate Testset
res = model.evaluate(ar_train_data_X, ar_train_data_Y, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = ar_test_data_X
yhat = model.predict(xhat)
testY = ar_test_data_Y.to_numpy()
yhat = back_MinMax(yhat, testY_max, testY_min)
testY = back_MinMax(testY, testY_max, testY_min)

xhat = ar_test_data_X
yhat1 = model1.predict(xhat)
testY = ar_test_data_Y.to_numpy()
yhat1 = back_MinMax(yhat1, testY_max, testY_min)
testY = back_MinMax(testY, testY_max, testY_min)

xhat = ar_test_data_X
yhat2 = model2.predict(xhat)
testY = ar_test_data_Y.to_numpy()
yhat2 = back_MinMax(yhat2, testY_max, testY_min)
testY = back_MinMax(testY, testY_max, testY_min)

print("test Y : ", testY, testY.shape,type(testY))
print("yhat : ",yhat, yhat.shape,type(yhat))

print("RMSE : {}".format(np.sqrt(np.average((yhat - testY)**2))))

In [ ]:
estimated = yhat
estimated1 = yhat1
estimated2 = yhat2
real = testY
real = real.T
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by MLP , Sigmoid")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(estimated1[:len(estimated1)],label = "Estimated by MLP , ReLU")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(estimated2[:len(estimated2)],label = "Estimated by MLP , Linear")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

In [ ]:
#Data Pre-preocssing for Forecasting
def MinMaxScaler2(data, max, min):
  denom = max - min
  nume = data-min
  return nume/denom

#Forecasting
forecasting_time = 10

new1 = model.predict(ar_test_data_X_testify)
print("A : ",ar_test_data_X_testify)
new_feedback1 = new1[-1]
print("B :", new_feedback1)
new_feedback = new_feedback1
forecasted = []
def forecast(forecasting_time) : 
  global forecasted
  global new_feedback
  for i in range(0, forecasting_time) : 
    new = model.predict(new_feedback)
    new_feedback = new
    if i%10 == 0 : print(i)
    forecasted = np.append(forecasted, new)
  return np.array(forecasted)

forecasting_result = forecast(forecasting_time)


In [ ]:
print(forecasting_result.dtype)
print(forecasting_result.shape)
print(forecasting_result)
forecasting_result = back_MinMax(forecasting_result,testX_max, testX_min)
t_estimated = forecasting_result
print("REAL : ",real)

In [ ]:
real = ar_test_data_Y_testify_original
real = real[testify + 1: testify + forecasting_time+1]
print("Real : ",real.shape)
plt.figure()
plt.plot(real[0 : forecasting_time+1], label = "Real")
plt.legend(prop = {'size':10})

print("Estimated : ",t_estimated.shape)
plt.figure()
plt.plot(t_estimated,label = "Forecasted by MLP")
plt.legend(prop = {'size':10})


error = real - t_estimated
plt.figure()
plt.plot(error, label = "Error")
plt.legend(prop = {'size':10})
print(error)

In [ ]:
# This model is not that powerful for forecasting
# I think this model has 2 critical problems. first, quantity of data. second, using only 1 feature
# should find multi vibrate and time-series forecasting method. not estimating